In [1]:
import requests
import pandas as pd
import numpy as np
import time
from bs4 import BeautifulSoup
import warnings
from unidecode import unidecode

warnings.filterwarnings("ignore")

In [2]:
# função que extrai as info para cada anuncio
def get_content(propiedade, data = {}):

    aux = []
    content = propiedade.find("div",class_="content")
    classe = 'headContent'

    aux = content.find("div",class_=classe)
    aux = aux.find("h4").text.strip()

    if "comprar" in aux:
        aux = aux.replace("para comprar","").strip()
        data["TITULO"].append(aux)
        data["OFERTA"].append("COMPRA")
    elif "alugar" in aux:
        aux = aux.replace("para alugar","").strip()
        data["TITULO"].append(aux)
        data["OFERTA"].append("ALUGAR")

    try:
        try:
            aux = propiedade.find("div",class_="RegramesDeDesconto")
            aux = aux.find("h1").text.replace("R$","").replace(".","").replace("*","").strip().split("por")
            valor = aux[0].split(" ")[-1].strip()
            if valor.isnumeric():
                data["VALORES"].append(valor)
            else:
                aux2 = propiedade.find("div",class_="RegramesDeDesconto").find("p").find("span").text.strip()
                data["VALORES"].append(aux2.replace("R$","").replace(".","").split(" ")[-1].strip())
            data["VALORES_PROMO"].append(aux[1].strip())
        except:
            aux = propiedade.find("div", class_="RegraMesesDesconto")
            aux = aux.find("h1").text.replace("R$","").replace(".","").replace("*","").strip().split("por")
            data["VALORES"].append(aux[0].split(" ")[-1].strip())
            data["VALORES_PROMO"].append(aux[1].strip())
    except:
        aux = propiedade.find("div",class_="RegraSemDesconto")
        price = aux.find("span").text.replace(".","").replace("R$","").strip()
        if not price.isnumeric():
            # CORREÇÃO DO ERRO PERCEBIDO NA BASE DE DADOS, ALGUNS IMOVEIS
            # APRESENTAM COMPOSIÇÃO DIFERENTE DE "SPANS"
            # NECESSÁRIO SELECIONAR O SEGUNDO <SPAN>
            price = aux.find_all("span")[1].text.replace(".","").replace("R$","").strip()
        data["VALORES"].append(price)
        data["VALORES_PROMO"].append(price)
    try:
        street = content.find("div", class_ = "RuaContainer")
        street = street.find_all("span")
        street = [st.text.strip() for st in street]
        data["RUA/AV"].append(street[0])
        data["LOGRADOURO"].append(street[1])
    except:
        data["RUA/AV"].append("UNKNOWN")
        data["LOGRADOURO"].append("UNKNOWN")
    try:
        aux = content.find("div",class_= "Location")
        aux = aux.find("span").text.split(",")
        data["LOCATION"].append(aux[0].strip())
        aux = aux[1].split("-")
        data["CITY"].append(aux[0].strip())
        data["STATE"].append(aux[1].strip())
    except:
        data["LOCATION"].append("Unknown")
        data["CITY"].append("Porto Alegre")
        data["STATE"].append("RS")
    
    classe = "Details"
    aux = content.find("div",class_=classe)
    aux = aux.find_all("div")

    try:
        data["METRAGEM"].append(aux[0].text.replace("m²","").strip())
    except:
        data["METRAGEM"].append(0)

    try:
        tgt = None
        for img in range(len(aux)):
            if aux[img].find('img', alt = True)['alt'] == "Quartos":
                tgt = img
        data["QUARTO"].append(aux[tgt].text)
    except:
        data["QUARTO"].append(0)

    try:
        tgt = None
        for img in range(len(aux)):
            if aux[img].find('img', alt = True)['alt'] == "Garagens":
                tgt = img
        data["GARAGEM"].append(aux[tgt].text)
    except:
        data["GARAGEM"].append(0)

    try:
        tgt = None
        for img in range(len(aux)):
            if aux[img].find('img', alt = True)['alt'] == "Banheiros":
                tgt = img
        data["BANHEIRO"].append(aux[tgt].text)
    except:
        data["BANHEIRO"].append(0)

    try:
        tag = []
        classe = "tags"
        aux = content.find("div", class_ = classe)
        aux = aux.find_all("h1")
        for tg in aux:
            tag.append(tg.text)
        data["TAG"].append("|".join(tag))
    except:
        data["TAG"].append([])

    classe = "footer"
    aux = content.find("div",class_ = classe)
    aux = aux.find_all("div")
    data["KEY"].append(aux[1].text.replace("ref: ",""))

In [3]:
import os

def get_data(
        date = "28-07-2023", op = "comprar", category = "residencial", file_name = "DatasetAux", i = 1
    ):
    file_name = date.replace("-","_") + "_" + file_name + "_" + category.capitalize()
    # URL PADRÃO DA AUXILIADORA PREDIAL
    url = 'https://www.auxiliadorapredial.com.br/' + op + '/' + category + '/rs+porto-alegre?page={}'
    # VARIAVEIS DE CONTROLE
    extraction, head = True, True
    if os.path.isfile(file_name + ".txt"):
        head = False
    # LOOP INDEFINIDO
    while extraction:
        # DICIONARIO DE VARIAVEIS DE INTERESSE
        data = {
            "TITULO": [],
            "OFERTA": [],
            "VALORES": [],
            "VALORES_PROMO": [],
            "RUA/AV": [],
            "LOGRADOURO": [],
            "LOCATION": [],
            "CITY": [],
            "STATE": [],
            "METRAGEM": [],
            "QUARTO": [],
            "GARAGEM": [],
            "BANHEIRO": [],
            "TAG": [],
            "KEY": []
        }

        try:
            # SCRAP DAS INFO DO SITE
            pagina = url.format(i)
            site = requests.get(pagina)
            soup = BeautifulSoup(site.text, "html.parser")
            property_listings = soup.find_all("div", class_="sc-c86ab3c9-0 fRIVEM")

            for property in property_listings:
                # INFORMAÇÕES DE UMA PÁGINA EM SI
                get_content(property, data = data)

            # PREPARA O DATAFRAME
            dataset = pd.DataFrame.from_dict(data, orient="index").transpose()
            #dataset["REF_DATE"] = date
            # MANDA PRA ARQUIVO AUXILIAR 
            dataset.to_csv(file_name + ".txt", index = False, header = head, mode = "a")
            print("Dados extraídos, página: ", i)

            # ATUALIZA A RODAGEM
            i += 1
            head = False
            time.sleep(0.1)

            # CONDIÇÃO DE TÉRMINO
            if dataset.empty:
                extraction = False

        except Exception as exc:
            # ROLOU ALGUMA MERDA
            extraction = False
            print(exc)

In [7]:
date = "2023-08-26"
# EXECUÇÃO DO ALGORITMO DE SCRAPING
get_data(date = date, op = "comprar", file_name = "Dataset_AP_Comprar", category = "residencial")
get_data(date = date, op = "alugar", file_name = "Dataset_AP_Alugar", category = "residencial")

get_data(date = date, op = "comprar", file_name = "Dataset_AP_Comprar", category = "comercial")
get_data(date = date, op = "alugar", file_name = "Dataset_AP_Alugar", category = "comercial")

HTTPSConnectionPool(host='www.auxiliadorapredial.com.br', port=443): Max retries exceeded with url: /comprar/residencial/rs+porto-alegre?page=1 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001F5EBB8DE50>: Failed to resolve 'www.auxiliadorapredial.com.br' ([Errno 11001] getaddrinfo failed)"))
HTTPSConnectionPool(host='www.auxiliadorapredial.com.br', port=443): Max retries exceeded with url: /alugar/residencial/rs+porto-alegre?page=1 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001F5EBBF0CD0>: Failed to resolve 'www.auxiliadorapredial.com.br' ([Errno 11001] getaddrinfo failed)"))
HTTPSConnectionPool(host='www.auxiliadorapredial.com.br', port=443): Max retries exceeded with url: /comprar/comercial/rs+porto-alegre?page=1 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001F5EBB24D30>: Failed to resolve 'www.auxiliadorapredial.com.br' ([Errno 11001] getaddrinfo failed)"))
HTTPSConne

In [5]:
def get_data_in_shape(date, file_name):

    def get_unique_tags(tgs):
        aux, ntags = [], []
        for c in tgs.columns:
            aux += tgs[c].unique().tolist()
        aux = list(set(aux))
        for t in aux:
            if type(t) == str: ntags.append(t)
        return(sorted(ntags))
    
    def cols_buy(cls):
        aux = []
        for c in cls:
            c2 = unidecode(c).upper().replace(" ","_")
            if c2 == "METRAGEM": c2 = "AREA"
            if c2 == "TITULO": c2 = "TIPO"
            aux.append(c2)
        return(aux)

    df = pd.read_csv(date.replace("-","_") + "_" + file_name+ ".txt")
    # TRATAMENTO DAS TAGS
    tags = df["TAG"].str.split("|", expand=True)
    new_tags = pd.DataFrame(0, index = tags.index, columns = get_unique_tags(tgs = tags))
    for i in tags.index:
        features = tags.loc[i,:].dropna().tolist()
        for feat in features:
            new_tags.loc[i,feat] = 1
    df.drop(labels = ["TAG"], axis = 1, inplace = True)
    df = pd.concat([df, new_tags], axis = 1)
    if "compra" in file_name.lower() or "aluga" in file_name.lower():
        # TRATAMENTO COMRPA
        df["VALORES"] = df["VALORES"].astype(int)
        df["VALORES_PROMO"] = df["VALORES_PROMO"].astype(int)
        df["VALORES"][df.VALORES < 100] = np.nan
        df["VALORES_PROMO"][df.VALORES_PROMO < 100] = np.nan
        df["METRAGEM"][df.METRAGEM <= 0] = np.nan
        df["LOCATION"] = df["LOCATION"].apply(lambda x: unidecode(x).strip().upper())
        df["TITULO"] = df["TITULO"].apply(lambda x: unidecode(x).strip().upper())
        df["CITY"] = df["CITY"].apply(lambda x: unidecode(x).strip().upper())
        df["LOGRADOURO"] = df["LOGRADOURO"].apply(lambda x: unidecode(x).strip().upper())
        # TRATAMENTO RUAS
        df["RUA/AV"] = df["RUA/AV"].str.replace(".","")
        df["RUA/AV"][df["RUA/AV"].isin(["rua", "rua doutor", "rua professor"])] = "rua"
        df["RUA/AV"][df["RUA/AV"].isin(["al", "alameda"])] = "alameda"
        df["RUA/AV"][df["RUA/AV"].isin(["av", "avenida"])] = "avenida"
        df["RUA/AV"][df["RUA/AV"].isin(["trav", "travessa"])] = "travessa"
        df["RUA/AV"][df["RUA/AV"].isin(["est", "estrada"])] = "estrada"
        df["RUA/AV"] = df["RUA/AV"].apply(lambda x: unidecode(x).strip().upper())
        df.columns = cols_buy(cls = df.columns.tolist())
        df["DATA_ANUNCIO"] = date
        df["DATA_RETIRADA"] = "-"
        return(df)

df_cp_rs = get_data_in_shape(date = date, file_name = "Dataset_AP_Comprar_Residencial")
df_al_rs = get_data_in_shape(date = date, file_name = "Dataset_AP_Alugar_Residencial")
df_cp_cc = get_data_in_shape(date = date, file_name = "Dataset_AP_Comprar_Comercial")
df_al_cc = get_data_in_shape(date = date, file_name = "Dataset_AP_Alugar_Comercial")

In [6]:
def merge_dataset(df, dataset):

    try:

        original = pd.read_csv(dataset + ".txt")

        dimension = original.shape[0]

        new_keys = df["KEY"].tolist()

        original[~original.KEY.isin(new_keys)]["DATA_RETIRADA"] = date
        original = pd.concat([original, df], axis = 0)

        original.drop_duplicates(
            subset=[
                'TIPO', 'OFERTA', 'VALORES', 'VALORES_PROMO', 'RUA/AV', 'LOGRADOURO',
                'LOCATION', 'CITY', 'STATE', 'AREA', 'QUARTO', 'GARAGEM', 'BANHEIRO',
                'KEY', 'DATA_RETIRADA'
            ], inplace = True
        )

        if original.shape[0] > dimension:
            original.to_csv(dataset + ".txt", index = False)

    except FileNotFoundError:
        df.to_csv(dataset + ".txt", index = False)

merge_dataset(df = df_cp_rs, dataset = "Dataset_AP_Comprar_Residencial")
merge_dataset(df = df_al_rs, dataset = "Dataset_AP_Alugar_Residencial")
merge_dataset(df = df_cp_cc, dataset = "Dataset_AP_Comprar_Comercial")
merge_dataset(df = df_al_cc, dataset = "Dataset_AP_Alugar_Comercial")

In [18]:
"""
import pandas

df1 = pandas.read_csv("Dataset_AP_Comprar_Residencial.txt")
df1["ADDRESS"] = df1["RUA/AV"] + " " + df1["LOGRADOURO"] + ", " + df1["CITY"]
df1["BAIRRO"] = df1["LOCATION"] + ", " + df1["CITY"]

df2 = pandas.read_csv("Dataset_AP_Comprar_Comercial.txt")
df2["ADDRESS"] = df2["RUA/AV"] + " " + df2["LOGRADOURO"] + ", " + df2["CITY"]
df2["BAIRRO"] = df2["LOCATION"] + ", " + df2["CITY"]

df3 = pandas.read_csv("Dataset_AP_Alugar_Residencial.txt")
df3["ADDRESS"] = df3["RUA/AV"] + " " + df3["LOGRADOURO"] + ", " + df3["CITY"]
df3["BAIRRO"] = df3["LOCATION"] + ", " + df3["CITY"]

df4 = pandas.read_csv("Dataset_AP_Alugar_Comercial.txt")
df4["ADDRESS"] = df4["RUA/AV"] + " " + df4["LOGRADOURO"] + ", " + df4["CITY"]
df4["BAIRRO"] = df4["LOCATION"] + ", " + df4["CITY"]


streets = pandas.concat([
    df1[["BAIRRO", "ADDRESS", "LAT_STREET", "LON_STREET"]],
    df2[["BAIRRO", "ADDRESS", "LAT_STREET", "LON_STREET"]],
    df3[["BAIRRO", "ADDRESS", "LAT_STREET", "LON_STREET"]],
    df4[["BAIRRO", "ADDRESS", "LAT_STREET", "LON_STREET"]]
    ], axis = 0
)

processing_streets = pandas.DataFrame(streets["ADDRESS"].unique().tolist())
processing_streets
"""

,BAIRRO,ADDRESS,LAT_STREET,LON_STREET
0,"BELA VISTA, PORTO ALEGRE","RUA JARAGUA, PORTO ALEGRE",-29.9693916,-51.6079471
1,"HIGIENOPOLIS, PORTO ALEGRE","RUA ARI MARINHO, PORTO ALEGRE",-30.0132564,-51.1796773
2,"MOINHOS DE VENTO, PORTO ALEGRE","RUA MARQUES DO HERVAL, PORTO ALEGRE",-30.0211299,-51.066669
3,"JARDIM SABARA, PORTO ALEGRE","RUA JOAO ERNESTO SCHMIDT, PORTO ALEGRE",-30.0347512,-51.1468274
4,"AZENHA, PORTO ALEGRE","AVENIDA FLORIANOPOLIS, PORTO ALEGRE",-30.0544875,-51.2133384
...,...,...,...,...
1901,"CENTRO HISTORICO, PORTO ALEGRE","RUA DOS ANDRADAS, PORTO ALEGRE",NaN,NaN
1902,"PRAIA DE BELAS, PORTO ALEGRE","AVENIDA PADRE CACIQUE, PORTO ALEGRE",NaN,NaN
1903,"PRAIA DE BELAS, PORTO ALEGRE","AVENIDA PADRE CACIQUE, PORTO ALEGRE",NaN,NaN
1904,"AUXILIADORA, PORTO ALEGRE","RUA 24 DE OUTUBRO, PORTO ALEGRE",NaN,NaN


In [7]:
from geopy.geocoders import Nominatim
import os

def street_lat_long():
    geolocator = Nominatim(user_agent="geoapiExercises")
    for f in os.listdir():
        if f.startswith("Dataset_"):
            df = pd.read_csv(f)
            if "LAT_STREET" not in df.columns:
                df["LAT_STREET"] = "-"
                df["LON_STREET"] = "-"
            df["ADDRESS"] = df["RUA/AV"] + " " + df["LOGRADOURO"] + ", " + df["CITY"]
            list_address = df["ADDRESS"].unique().tolist()
            for add in list_address:
                try:
                    location = geolocator.geocode(add)
                    df["LAT_STREET"][(df.LAT_STREET == "-") & (df.ADDRESS == add)] = location.latitude
                    df["LON_STREET"][(df.LON_STREET == "-") & (df.ADDRESS == add)] = location.longitude
                except:
                    df["LAT_STREET"][df.ADDRESS == add] = "-"
                    df["LON_STREET"][df.ADDRESS == add] = "-"
            df.drop("ADDRESS", axis = 1, inplace = True)
            df.to_csv(f, index = False)

street_lat_long()

In [ ]:
import geobr

geobr.read_neighborhood()